## Import necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='keras')

## Data Augmentation and Normalization for training and validation data

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    shear_range=0.2,  # shear transformations
    zoom_range=0.2,   # random zoom
    horizontal_flip=True,  # horizontal flips
    validation_split=0.2  # Set 20% of training data as validation data
)

# Rescale validation data
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    r'C:\Users\Admin\Documents\Jupyter Notebook\AI Project\garbage_classification',
    target_size=(128, 128),
    batch_size=64,  
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\Admin\Documents\Jupyter Notebook\AI Project\garbage_classification',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

Found 12414 images belonging to 12 classes.
Found 3100 images belonging to 12 classes.


In [12]:
# Load the VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the VGG16 base layers so they don't get updated during training
for layer in base_model.layers:
    layer.trainable = False

## Build a new model

In [13]:
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Flatten the output of the VGG16 model
model.add(Flatten())

# Add a Dense layer
model.add(Dense(256, activation='relu'))

# Add a Dropout layer (to reduce overfitting)
model.add(Dropout(0.5))

# Add the final Dense layer (for classification)
model.add(Dense(12, activation='softmax'))

## Compile the model with updated optimizer

In [14]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

## Train the model

In [15]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

Epoch 1/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 328s 2s/step - accuracy: 0.5036 - loss: 1.6272 - val_accuracy: 0.6987 - val_loss: 0.9137
Epoch 2/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.7547 - loss: 0.8041 - val_accuracy: 0.7581 - val_loss: 0.7428
Epoch 3/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8073 - loss: 0.6355 - val_accuracy: 0.7761 - val_loss: 0.6774
Epoch 4/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8178 - loss: 0.5828 - val_accuracy: 0.8029 - val_loss: 0.6039
Epoch 5/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.8434 - loss: 0.5114 - val_accuracy: 0.8094 - val_loss: 0.5834
Epoch 6/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.8519 - loss: 0.4794 - val_accuracy: 0.8239 - val_loss: 0.5508
Epoch 7/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.8645 - loss: 0.4378 - val_accuracy: 0.8171 - val_loss: 0.5475
Epoch 8/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - accuracy: 0.8670 - loss: 0.4191 - val_accu

## Evaluate the model on validation data

In [21]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy*100:.2f}%')

49/49 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.8616 - loss: 0.4408
Validation Accuracy: 86.10%


## Save the trained model

In [17]:
model.save('Eco_Bin.h5')

## Load the saved model and Predict on new Image

In [20]:
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from PIL import Image

# Load the model
model = load_model('Eco_Bin.h5')

# Define the class labels
class_labels = [
    'battery', 'biological', 'brown-glass', 'cardboard', 'clothes',
    'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass'
]

# Predict on a new image
img_path = r'C:\Users\Admin\Documents\Jupyter Notebook\AI Project\Test Set\snake.jpg' 

# Preprocess the image
img = Image.open(img_path)
img = img.resize((128, 128))  

# Check if the image is grayscale, (and convert to RGB if needed)
if img.mode != "RGB":
    img = img.convert("RGB")

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Get the predicted class index
prediction = model.predict(img_array)
predicted_class_index = np.argmax(prediction, axis=1)[0]

# Map the index to the class name using class_labels
predicted_class_name = class_labels[predicted_class_index]

# Print the predicted class name
print(f'The Category of the Object is: {predicted_class_name}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
The Category of the Object is: green-glass
